In [ ]:
#| hide
!pip install -Uqq nixtla

In [ ]:
#| hide 
from nixtla.utils import in_colab

In [ ]:
#| hide 
IN_COLAB = in_colab()

In [ ]:
#| hide
if not IN_COLAB:
    from nixtla.utils import colab_badge
    from dotenv import load_dotenv

# Irregular timestamps

* For pandas dataframes TimeGPT will try to infer the frequency of your timestamps. If you don't want TimeGPT to infer the frequency for you, you have to set the  `freq` argument to a valid [pandas frequency string](https://pandas.pydata.org/docs/user_guide/timeseries.html#dateoffset-objects), e.g. 'MS', 'YE'.
* For polars dataframes you have to set the `freq` argument to a valid [polars offset](https://docs.pola.rs/api/python/stable/reference/expressions/api/polars.Expr.dt.offset_by.html), e.g. '1d', '2w'.

If your data has missing timestamps please refer to our [tutorial on missing values](https://docs.nixtla.io/docs/tutorials-missing_values).

In [ ]:
#| echo: false
if not IN_COLAB:
    load_dotenv()
    colab_badge('docs/capabilities/forecast/11_irregular_timestamps')

[![](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Nixtla/nixtla/blob/main/nbs/docs/capabilities/forecast/11_irregular_timestamps.ipynb)

In [ ]:
import pandas as pd
from nixtla import NixtlaClient
from utilsforecast.data import generate_series

In [ ]:
nixtla_client = NixtlaClient(
    # defaults to os.environ.get("NIXTLA_API_KEY")
    api_key = 'my_api_key_provided_by_nixtla'
)

> 👍 Use an Azure AI endpoint
> 
> To use an Azure AI endpoint, remember to set also the `base_url` argument:
> 
> `nixtla_client = NixtlaClient(base_url="you azure ai endpoint", api_key="your api_key")`

In [ ]:
#| hide
if not IN_COLAB:
    nixtla_client = NixtlaClient()

In [ ]:
#| hide
_ = nixtla_client._get_model_params('timegpt-1', 'B')

INFO:nixtla.nixtla_client:Querying model metadata...


In [ ]:
# Read data
# Dates for the weekends are missing
df = pd.read_csv(
    'https://datasets-nixtla.s3.amazonaws.com/pltr.csv',
    usecols=['date', 'Close'],
)

# Forecast
# the frequency is inferred as B, as only business days are represented in the dataset
forecast_df = nixtla_client.forecast(
    df=df,
    h=5,
    time_col='date', 
    target_col='Close',
)

INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Inferred freq: B
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...


In [ ]:
# manually set the frequency
forecast_df2 = nixtla_client.forecast(
    df=df,
    freq='B',
    h=5,
    time_col='date', 
    target_col='Close',
)

INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...


In [ ]:
pd.testing.assert_frame_equal(forecast_df, forecast_df2)

> 📘 Available models in Azure AI
>
> If you are using an Azure AI endpoint, please be sure to set `model="azureai"`:
>
> `nixtla_client.forecast(..., model="azureai")`
> 
> For the public API, we support two models: `timegpt-1` and `timegpt-1-long-horizon`. 
> 
> By default, `timegpt-1` is used. Please see [this tutorial](https://docs.nixtla.io/docs/tutorials-long_horizon_forecasting) on how and when to use `timegpt-1-long-horizon`.